# Text annotation 


In [ ]:
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from pandas.io.json import json_normalize
import requests
import beakerx
from pynanomapper import aa



In [ ]:
def search_service_protected(url,apikey):
    return (url,apikey)
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

In [ ]:
service_uri=service_widget.value
print("Sending queries to {}".format(service_uri))
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)


In [ ]:
options = {'url': service_uri + '/annotate?overlaps=ALL&tagsLimit=5000','name':'','password':''}


In [ ]:
# example query excerpt from https://arxiv.org/abs/1906.00212
query = "HRTEM, with EDX and high-angle annular dark-field scanning transmission electron microscopy were performed using \
JEOL JEMARM200F and JEM-ARM200CF with an acceleration voltage of 200 kV. The JEOL Analysis Station software was used to control the STEM-EDX mapping. \
To minimize the effect of sample drift, a driftcorrection mode was used during acquisition of the elemental map.\
The STEM probe size was ~0.13 nm, generating ~140 pA of current when 40 \u00b5m of the condenser lens aperture was inserted. \
The collection angle of the HAADF detector was ~97-256 mr"

defcolumn = "definition"
@interact(query=query)
def g(query):
    x = requests.post(options['url'], data = query.encode(encoding='utf-8'),  auth=auth_object)
    if x.status_code==200:
        #print(x.json())
        tags=x.json()["tags"]
        results = json_normalize(x.json()['response']['docs'])
        for tag in tags:
            #print(tag)
            _id=tag[5][0]
            term = results[results["id"].isin(tag[5])]      
            print("{}\t [at offset {}:{}]".format(query[tag[1]:tag[3]],tag[1],tag[3]))
            c = ["id","name","ontology"]
            if defcolumn in term.columns.values:
                c.extend([defcolumn])
            display(term[c])

        #display(results)
        #print(x.json())
    else:

        display(x.status_code)  
    return (query)



In [ ]:
# tbd create enm ontology github issue  automatically 
# https://gist.github.com/JeffPaine/3145490


In [ ]:
%%javascript
var response="{'responseHeader': {'status': 0, 'QTime': 0}, 'tagsCount': 2, 'tags': [['startOffset', 0, 'endOffset', 4, 'ids', ['EFO_0001086']], ['startOffset', 5, 'endOffset', 10, 'ids', ['EFO_0001099']]], 'response': {'numFound': 2, 'start': 0, 'docs': [{'Parents': ['http://purl.obolibrary.org/obo/BTO_0001616', 'http://www.ebi.ac.uk/efo/EFO_0002926', 'http://purl.obolibrary.org/obo/BTO_0000797'], 'ontology': ['EFO'], 'id': 'EFO_0001099', 'prefix': ['http://www.ebi.ac.uk/efo'], 'name': ['Caco-2', 'Caco2'], '_version_': 1643924168342765576}, {'Parents': ['http://www.ebi.ac.uk/efo/EFO_0002888', 'http://www.ebi.ac.uk/efo/EFO_0002934'], 'ontology': ['EFO'], 'id': 'EFO_0001086', 'prefix': ['http://www.ebi.ac.uk/efo'], 'name': ['A549', 'A-549', 'A549 cell'], '_version_': 1643924168630075394}]}}"

var ranges = $.map( response["tags"], function( val, i ) {
  return val[0];
});


element.text(ranges.join())